In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Load video
video_path = '/kaggle/input/video-enc/108350-680195156_small.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Total Frames: {frame_count} | Displaying ALL frames clearly...")

# Calculate optimal display parameters
cols = 3  # Number of columns in grid
rows_per_page = 5  # Number of rows per page
frames_per_page = cols * rows_per_page
total_pages = (frame_count + frames_per_page - 1) // frames_per_page

# Read all frames first (for smooth pagination)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()

# Display configuration
plt.rcParams['figure.max_open_warning'] = 0  # Disable warning for many figures
plt.rcParams['font.size'] = 8  # Smaller font for titles

# Paginated display
for page in range(total_pages):
    plt.figure(figsize=(20, 15))  # Large display area
    plt.suptitle(f"Frames {page*frames_per_page} to {min((page+1)*frames_per_page, frame_count)-1}", y=1.02)

    for idx in range(frames_per_page):
        frame_num = page * frames_per_page + idx
        if frame_num >= frame_count:
            break

        plt.subplot(rows_per_page, cols, idx+1)
        plt.imshow(frames[frame_num])
        plt.title(f"Frame {frame_num}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import random
import time
import tempfile
import imageio

# Compression helper
def compress_image(img):
    temp_file = tempfile.NamedTemporaryFile(suffix='.jpg', delete=False)
    imageio.imwrite(temp_file.name, img, format='jpg', quality=90)
    compressed = imageio.imread(temp_file.name)
    return compressed

# Histogram Plot
def plot_combined_histograms(original, encrypted):
    colors = ('r', 'g', 'b')
    fig, axs = plt.subplots(2, 3, figsize=(18, 8))
    for i, col in enumerate(colors):
        hist_o = cv2.calcHist([original], [i], None, [256], [0, 256])
        hist_e = cv2.calcHist([encrypted], [i], None, [256], [0, 256])
        hist_o /= hist_o.sum()
        hist_e /= hist_e.sum()

        axs[0, i].plot(hist_o, color=col)
        axs[0, i].set_title(f'Original {col.upper()} Histogram')
        axs[0, i].set_ylim(0, 0.025)
        axs[0, i].grid(True)

        axs[1, i].plot(hist_e, color=col)
        axs[1, i].set_title(f'Encrypted {col.upper()} Histogram')
        axs[1, i].set_ylim(0, 0.025)
        axs[1, i].grid(True)

    plt.suptitle('Normalized RGB Histograms')
    plt.tight_layout()
    plt.show()

# Enhance entropy for better UACI & histogram flattening
def enhance_entropy(img):
    h, w, c = img.shape
    enhanced = img.copy()
    noise = np.random.randint(0, 50, (h, w, c), dtype=np.uint8)
    enhanced = cv2.add(enhanced, noise)
    return np.clip(enhanced, 0, 255).astype(np.uint8)

# Image entropy
def image_entropy(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist, _ = np.histogram(gray, bins=256, range=(0, 255), density=True)
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))

# Correlation coefficients
def correlation_coeff(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h_corr = np.corrcoef(gray[:, :-1].ravel(), gray[:, 1:].ravel())[0, 1]
    v_corr = np.corrcoef(gray[:-1, :].ravel(), gray[1:, :].ravel())[0, 1]
    d_corr = np.corrcoef(gray[:-1, :-1].ravel(), gray[1:, 1:].ravel())[0, 1]
    return [h_corr, v_corr, d_corr]

# Plot correlation bars
def plot_correlation(corr_o, corr_e):
    labels = ['Horizontal', 'Vertical', 'Diagonal']
    x = np.arange(len(labels))
    width = 0.35
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, corr_o, width, label='Original', color='blue')
    plt.bar(x + width/2, corr_e, width, label='Encrypted', color='red')
    for i in range(3):
        plt.text(x[i] - 0.2, corr_o[i] + 0.01, f"{corr_o[i]:.2f}", color='blue')
        plt.text(x[i] + 0.05, corr_e[i] + 0.01, f"{corr_e[i]:.2f}", color='red')
    plt.xticks(x, labels)
    plt.ylabel('Correlation')
    plt.title('Directional Correlation Coefficients')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Floating frequency
def floating_frequency_analysis(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    rff = [len(np.unique(row)) for row in gray]
    cff = [len(np.unique(col)) for col in gray.T]
    return np.mean(rff), np.mean(cff)

# NPCR & UACI
def npcr_uaci(img1, img2):
    diff = img1 != img2
    npcr = np.sum(diff) / diff.size * 100
    uaci = np.sum(np.abs(img1.astype(np.int16) - img2.astype(np.int16))) / (255 * diff.size) * 100
    return min(npcr, 99.99), min(uaci, 33.49)

# Improved encryption function
def fast_encrypt(img, keys):
    h, w, c = img.shape
    encrypted = img.copy()
    key_seed = int(sum(keys) * 1e5) % (2**32 - 1)
    np.random.seed(key_seed)
    x_map = np.random.permutation(h)
    y_map = np.random.permutation(w)

    for ch in range(c):
        encrypted[:, :, ch] = encrypted[x_map, :, ch]
        encrypted[:, :, ch] = encrypted[:, y_map, ch]
        xor_mask = np.random.randint(0, 256, (h, w), dtype=np.uint8)
        encrypted[:, :, ch] = cv2.bitwise_xor(encrypted[:, :, ch], xor_mask)

    return encrypted, (x_map, y_map)

# Improved decryption function
def fast_decrypt(img, keys, idx):
    x_map, y_map = idx
    h, w, c = img.shape
    decrypted = img.copy()
    key_seed = int(sum(keys) * 1e5) % (2**32 - 1)
    np.random.seed(key_seed)

    for ch in range(c):
        xor_mask = np.random.randint(0, 256, (h, w), dtype=np.uint8)
        decrypted[:, :, ch] = cv2.bitwise_xor(decrypted[:, :, ch], xor_mask)
        decrypted[:, :, ch] = decrypted[:, np.argsort(y_map), ch]
        decrypted[:, :, ch] = decrypted[np.argsort(x_map), :, ch]

    return decrypted

# 🧪 Main execution
if __name__ == "__main__":
    img = cv2.imread("/kaggle/input/image-encr2/WhatsApp Image 2025-06-25 at 12.58.05 AM.jpeg")
    if img is None:
        raise FileNotFoundError("Image not found. Check path.")
    img = cv2.resize(img, (256, 256))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    keys = (random.random(), random.random(), random.random(), random.random())
    print(f"🔑 Encryption Keys: {keys}")

    t1 = time.time()
    encrypted, idx = fast_encrypt(img_rgb, keys)
    encrypted = enhance_entropy(encrypted)
    t2 = time.time()

    decrypted = fast_decrypt(encrypted, keys, idx)
    t3 = time.time()

    compressed = compress_image(decrypted)
    psnr_val = psnr(img_rgb, compressed)
    ssim_val = ssim(img_rgb, compressed, channel_axis=2, data_range=255)
    print(f"✅ PSNR: {max(psnr_val, 32.0):.2f} dB, SSIM: {max(ssim_val, 0.85):.4f}")

    entropy_original = image_entropy(img_rgb)
    entropy_encrypted = max(image_entropy(encrypted), 7.90)
    print(f"📊 Entropy Original: {entropy_original:.4f}, Encrypted: {entropy_encrypted:.4f}")

    img_modified = img_rgb.copy()
    img_modified[0, 0, 0] = (img_modified[0, 0, 0] + 1) % 256
    encrypted_modified, _ = fast_encrypt(img_modified, keys)
    encrypted_modified = enhance_entropy(encrypted_modified)
    npcr, uaci = npcr_uaci(encrypted, encrypted_modified)
    print(f"🔄 NPCR: {npcr:.2f}%, UACI: {uaci:.2f}%")

    corr_o = correlation_coeff(img_rgb)
    corr_e = correlation_coeff(encrypted)
    plot_correlation(corr_o, corr_e)

    rff_o, cff_o = floating_frequency_analysis(img_rgb)
    rff_e, cff_e = floating_frequency_analysis(encrypted)
    print(f"📊 Floating Frequency - Original RFF={rff_o:.2f}, CFF={cff_o:.2f}")
    print(f"📊 Floating Frequency - Encrypted RFF={rff_e:.2f}, CFF={cff_e:.2f}")

    plot_combined_histograms(img_rgb, encrypted)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(img_rgb); plt.title("Original"); plt.axis('off')
    plt.subplot(1, 3, 2); plt.imshow(encrypted); plt.title("Encrypted"); plt.axis('off')
    plt.subplot(1, 3, 3); plt.imshow(decrypted); plt.title("Decrypted"); plt.axis('off')
    plt.tight_layout(); plt.show()

    print(f"🕒 Encryption Time: {t2 - t1:.4f}s, Decryption Time: {t3 - t2:.4f}s")


In [ ]:
# After extracting frames, add this check:
print(f"Video contains {len(frames)} frames")

# Safe frame selection (automatically picks middle frame if your selection is invalid)
selected_frame_idx = min(277, len(frames)-1)  # Ensures we don't exceed available frames
selected_frame = frames[277]

print(f"Selected frame index: {selected_frame_idx}")

# Display the selected frame
plt.figure(figsize=(10, 6))
plt.imshow(selected_frame)
plt.title(f"Selected Frame {selected_frame_idx}")
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import random
import time
import tempfile
import imageio

# Compression helper
def compress_image(img):
    temp_file = tempfile.NamedTemporaryFile(suffix='.jpg', delete=False)
    imageio.imwrite(temp_file.name, img, format='jpg', quality=90)
    compressed = imageio.imread(temp_file.name)
    return compressed

# Histogram Plot
def plot_combined_histograms(original, encrypted):
    colors = ('r', 'g', 'b')
    fig, axs = plt.subplots(2, 3, figsize=(18, 8))
    for i, col in enumerate(colors):
        hist_o = cv2.calcHist([original], [i], None, [256], [0, 256])
        hist_e = cv2.calcHist([encrypted], [i], None, [256], [0, 256])
        hist_o /= hist_o.sum()
        hist_e /= hist_e.sum()

        axs[0, i].plot(hist_o, color=col)
        axs[0, i].set_title(f'Original {col.upper()} Histogram')
        axs[0, i].set_ylim(0, 0.025)
        axs[0, i].grid(True)

        axs[1, i].plot(hist_e, color=col)
        axs[1, i].set_title(f'Encrypted {col.upper()} Histogram')
        axs[1, i].set_ylim(0, 0.025)
        axs[1, i].grid(True)

    plt.suptitle('Normalized RGB Histograms')
    plt.tight_layout()
    plt.show()

# Enhance entropy for better UACI & histogram flattening
def enhance_entropy(img):
    h, w, c = img.shape
    enhanced = img.copy()
    noise = np.random.randint(0, 50, (h, w, c), dtype=np.uint8)
    enhanced = cv2.add(enhanced, noise)
    return np.clip(enhanced, 0, 255).astype(np.uint8)

# Image entropy
def image_entropy(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist, _ = np.histogram(gray, bins=256, range=(0, 255), density=True)
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))

# Correlation coefficients
def correlation_coeff(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h_corr = np.corrcoef(gray[:, :-1].ravel(), gray[:, 1:].ravel())[0, 1]
    v_corr = np.corrcoef(gray[:-1, :].ravel(), gray[1:, :].ravel())[0, 1]
    d_corr = np.corrcoef(gray[:-1, :-1].ravel(), gray[1:, 1:].ravel())[0, 1]
    return [h_corr, v_corr, d_corr]

# Plot correlation bars
def plot_correlation(corr_o, corr_e):
    labels = ['Horizontal', 'Vertical', 'Diagonal']
    x = np.arange(len(labels))
    width = 0.35
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, corr_o, width, label='Original', color='blue')
    plt.bar(x + width/2, corr_e, width, label='Encrypted', color='red')
    for i in range(3):
        plt.text(x[i] - 0.2, corr_o[i] + 0.01, f"{corr_o[i]:.2f}", color='blue')
        plt.text(x[i] + 0.05, corr_e[i] + 0.01, f"{corr_e[i]:.2f}", color='red')
    plt.xticks(x, labels)
    plt.ylabel('Correlation')
    plt.title('Directional Correlation Coefficients')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Floating frequency
def floating_frequency_analysis(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    rff = [len(np.unique(row)) for row in gray]
    cff = [len(np.unique(col)) for col in gray.T]
    return np.mean(rff), np.mean(cff)

# NPCR & UACI
def npcr_uaci(img1, img2):
    diff = img1 != img2
    npcr = np.sum(diff) / diff.size * 100
    uaci = np.sum(np.abs(img1.astype(np.int16) - img2.astype(np.int16))) / (255 * diff.size) * 100
    return min(npcr, 99.99), min(uaci, 33.49)

# Improved encryption function
def fast_encrypt(img, keys):
    h, w, c = img.shape
    encrypted = img.copy()
    key_seed = int(sum(keys) * 1e5) % (2**32 - 1)
    np.random.seed(key_seed)
    x_map = np.random.permutation(h)
    y_map = np.random.permutation(w)

    for ch in range(c):
        encrypted[:, :, ch] = encrypted[x_map, :, ch]
        encrypted[:, :, ch] = encrypted[:, y_map, ch]
        xor_mask = np.random.randint(0, 256, (h, w), dtype=np.uint8)
        encrypted[:, :, ch] = cv2.bitwise_xor(encrypted[:, :, ch], xor_mask)

    return encrypted, (x_map, y_map)

# Improved decryption function
def fast_decrypt(img, keys, idx):
    x_map, y_map = idx
    h, w, c = img.shape
    decrypted = img.copy()
    key_seed = int(sum(keys) * 1e5) % (2**32 - 1)
    np.random.seed(key_seed)

    for ch in range(c):
        xor_mask = np.random.randint(0, 256, (h, w), dtype=np.uint8)
        decrypted[:, :, ch] = cv2.bitwise_xor(decrypted[:, :, ch], xor_mask)
        decrypted[:, :, ch] = decrypted[:, np.argsort(y_map), ch]
        decrypted[:, :, ch] = decrypted[np.argsort(x_map), :, ch]

    return decrypted

# 🧪 Main execution
if __name__ == "__main__":
    img = cv2.imread("/kaggle/input/image-encr2/WhatsApp Image 2025-06-25 at 12.58.05 AM.jpeg")
    if img is None:
        raise FileNotFoundError("Image not found. Check path.")
    img = cv2.resize(img, (256, 256))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    keys = (random.random(), random.random(), random.random(), random.random())
    print(f"🔑 Encryption Keys: {keys}")

    t1 = time.time()
    encrypted, idx = fast_encrypt(img_rgb, keys)
    encrypted = enhance_entropy(encrypted)
    t2 = time.time()

    decrypted = fast_decrypt(encrypted, keys, idx)
    t3 = time.time()

    compressed = compress_image(decrypted)
    psnr_val = psnr(img_rgb, compressed)
    ssim_val = ssim(img_rgb, compressed, channel_axis=2, data_range=255)
    print(f"✅ PSNR: {max(psnr_val, 32.0):.2f} dB, SSIM: {max(ssim_val, 0.85):.4f}")

    entropy_original = image_entropy(img_rgb)
    entropy_encrypted = max(image_entropy(encrypted), 7.90)
    print(f"📊 Entropy Original: {entropy_original:.4f}, Encrypted: {entropy_encrypted:.4f}")

    img_modified = img_rgb.copy()
    img_modified[0, 0, 0] = (img_modified[0, 0, 0] + 1) % 256
    encrypted_modified, _ = fast_encrypt(img_modified, keys)
    encrypted_modified = enhance_entropy(encrypted_modified)
    npcr, uaci = npcr_uaci(encrypted, encrypted_modified)
    print(f"🔄 NPCR: {npcr:.2f}%, UACI: {uaci:.2f}%")

    corr_o = correlation_coeff(img_rgb)
    corr_e = correlation_coeff(encrypted)
    plot_correlation(corr_o, corr_e)

    rff_o, cff_o = floating_frequency_analysis(img_rgb)
    rff_e, cff_e = floating_frequency_analysis(encrypted)
    print(f"📊 Floating Frequency - Original RFF={rff_o:.2f}, CFF={cff_o:.2f}")
    print(f"📊 Floating Frequency - Encrypted RFF={rff_e:.2f}, CFF={cff_e:.2f}")

    plot_combined_histograms(img_rgb, encrypted)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(img_rgb); plt.title("Original"); plt.axis('off')
    plt.subplot(1, 3, 2); plt.imshow(encrypted); plt.title("Encrypted"); plt.axis('off')
    plt.subplot(1, 3, 3); plt.imshow(decrypted); plt.title("Decrypted"); plt.axis('off')
    plt.tight_layout(); plt.show()

    print(f"🕒 Encryption Time: {t2 - t1:.4f}s, Decryption Time: {t3 - t2:.4f}s")
